<a href="https://colab.research.google.com/github/lagodw/RedditBot/blob/master/Reddit_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Sep  3 15:35:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
%%capture
!pip install transformers datasets

In [ ]:
#! usr/bin/env python3
import pandas as pd
import datetime as dt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw_text = pd.read_csv('/content/drive/MyDrive/reddit_scrape.csv')

raw_text.loc[raw_text.parent_text.isna(), 'parent_text'] = raw_text.loc[raw_text.parent_text.isna(), 'title']

raw_text['parent_text'] = raw_text.subreddit + '. ' + raw_text.parent_text

raw_text['comment_text'] = raw_text.comment_text.str.lower()
raw_text['parent_text'] = raw_text.parent_text.str.lower()
raw_text = raw_text[(raw_text.comment_text.notna()) & (raw_text.parent_text.notna())]

raw_text['total_length'] = raw_text.parent_text.str.len() + raw_text.comment_text.str.len()
raw_text = raw_text[raw_text.total_length < 1000]
raw_text = raw_text.sample(frac=1).reset_index(drop = True, random_state = 123)

raw_text['input'] = raw_text['parent_text'] + '<sep>' + raw_text['comment_text']

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
raw_text.head()

,subreddit,title,comment_text,parent_text,parent_id,comment_id,total_length,input
0,Coronavirus,"Anxiety, depression, poor mental health – our ...",aids is pandemic,coronavirus. there’s literally zero precedent ...,haoo2s5,haorl4b,99,coronavirus. there’s literally zero precedent ...
1,mildlyinteresting,Fashion has cycled back to the 1800s American ...,looks like crazy ralph's outfit from,mildlyinteresting. fashion has cycled back to ...,p8pqyp,h9sdnld,155,mildlyinteresting. fashion has cycled back to ...
2,FortNiteBR,Today was my Birthday (August 30th) so I got a...,![gif](giphy|ivtifdvulinia),fortnitebr. ![gif](giphy|feio2yiumtdqwjriaf|do...,hazv1x3,hb0k1mo,81,fortnitebr. ![gif](giphy|feio2yiumtdqwjriaf|do...
3,news,China will allow couples to have three childre...,"not in all places, it varies by state and ther...",news. in america the government can tell a 14 ...,h9wva4g,h9yw9gf,488,news. in america the government can tell a 14 ...
4,news,"Oregon Governor Tells Doctors, Nurses and Teac...",already happening *becaise*of feckless antivax...,"news. yep, but clearly many of them haven’t. a...",h9ls4r7,h9m706b,293,"news. yep, but clearly many of them haven’t. a..."


In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', 
                                          bos_token='<|startoftext|>', 
                                          eos_token='<|endoftext|>', 
                                          pad_token='<|pad|>',
                                          unk_token ='<unk>',
                                          sep_token ='<sep>',
                                          cls_token ='<cls>',
                                          mask_token='<mask>')


tokenizer.encode("Sample Text")

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[36674, 8255]

In [ ]:
from datasets import Dataset

raw_datasets = Dataset.from_pandas(raw_text[['input']])
del raw_text

def tokenize_function(examples, max_length = 252):

    model_inputs = tokenizer(examples["input"], padding='max_length', truncation=True, max_length=max_length, add_special_tokens=True)
    # with tokenizer.as_target_tokenizer():
    #   labels = tokenizer(examples["parent_text"], examples['comment_text'], padding='max_length', truncation=True, max_length=max_length, add_special_tokens=True)

    model_inputs['labels'] = model_inputs['input_ids']
    return model_inputs

tokenized_datasets = raw_datasets.map(tokenize_function)
tokenized_datasets = tokenized_datasets.remove_columns(['input'])
if '__index_level_0__' in tokenized_datasets.features:
  tokenized_datasets = tokenized_datasets.remove_columns(['__index_level_0__'])

del raw_datasets

  0%|          | 0/5062327 [00:00<?, ?ex/s]

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments

#model = GPT2LMHeadModel.from_pretrained('gpt2')

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir = 'redditbot',
    num_train_epochs = 2,
    per_device_train_batch_size = 16,
    warmup_steps = 100,
    weight_decay = 0.01,
    save_strategy = 'epoch'
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets
)

trainer.train()

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

***** Running training *****
  Num examples = 5062327
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 632792


Step,Training Loss
500,2.060500
1000,0.880000
1500,0.845100
2000,0.843100
2500,0.831400
3000,0.825500
3500,0.807100
4000,0.811100
4500,0.789900
5000,0.812900


In [ ]:
import torch
device = torch.device("cuda")

model.eval()

prompt = "<|startoftext|> askreddit."
prompt = prompt.lower()

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=50, 
                                max_length = 100,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  askreddit. if you could rename the word "popcorn" what would it be?.. potato,)


1:  askreddit. why do you think it’s ok to piss in the shower?. for someone who's never had a shower, it makes a ton of sense, no? i've been told to clean my own faucets once or twice. 

having a large tub with a large shower tub seems really wasteful.. to be fair, most of the time, it works well. my faucet just has been there for like 5-10 mins.


2:  askreddit. what are some reasons people give when they're attacked?.. being polite with people




In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 50 not upgraded.
Need to get 6,229 kB of archives.
After this operation, 14.5 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.13.3 [6,229 kB]
Fetched 6,229 kB in 0s (18.2 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


In [ ]:
! git config --global user.email "lagodw@gmail.com"

In [ ]:
tokenizer.push_to_hub("lagodw/redditbot", use_auth_token = 'api_hIRjMxMnWfMDJZLlqYtEmZUUUVzbofCfRV')
model.push_to_hub("lagodw/redditbot", use_auth_token = 'api_hIRjMxMnWfMDJZLlqYtEmZUUUVzbofCfRV')

tokenizer config file saved in lagodw/redditbot/tokenizer_config.json
Special tokens file saved in lagodw/redditbot/special_tokens_map.json
added tokens file saved in lagodw/redditbot/added_tokens.json
Configuration saved in lagodw/redditbot/config.json
Model weights saved in lagodw/redditbot/pytorch_model.bin


'https://huggingface.co/lagodw/redditbot/commit/72c28ef74178fb13aaa91e625136d65406253ca8'

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
test_tokenizer = AutoTokenizer.from_pretrained("lagodw/redditbot")

test_model = AutoModelWithLMHead.from_pretrained("lagodw/redditbot")

Didn't find file lagodw/redditbot/tokenizer.json. We won't load it.
loading file lagodw/redditbot/vocab.json
loading file lagodw/redditbot/merges.txt
loading file None
loading file lagodw/redditbot/added_tokens.json
loading file lagodw/redditbot/special_tokens_map.json
loading file lagodw/redditbot/tokenizer_config.json
Adding <|startoftext|> to the vocabulary
Adding <sep> to the vocabulary
Adding <|pad|> to the vocabulary
Adding <cls> to the vocabulary
Adding <mask> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:902: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configurati

In [ ]:
import torch
device = torch.device("cuda")

test_model.eval()

prompt = "<|startoftext|> askreddit. What is something you have, that your pretty sure no one else has?"
prompt = prompt.lower()

generated = torch.tensor(test_tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

test_model.to(device)

sample_outputs = test_model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=5, 
                                max_length = 100,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, test_tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  askreddit. what is something you have, that your pretty sure no one else has?.. i have a friend who’s been with me for about a year and we've been talking about how we love each other and how i don’t want to let go of him because i’m so happy to hear that he’s doing great and i’m happy to be able to share more with him.


1:  askreddit. what is something you have, that your pretty sure no one else has?.. not my favorite thing to do’) “what’s up, boys?”)”)


2:  askreddit. what is something you have, that your pretty sure no one else has?.. i have a pretty good idea of how to get a girl pregnant’s) “what is the word you want to say?”?


